# Introduction to US Federal Government Contracts

* *This notebook is part of the [Government Procurement Queries](https://github.com/antontarasenko/gpq) project*

## The Dataset

The BigQuery dataset (17 years of data, 45mn transactions, $6.7tn worth of goods and services):

- [gpqueries:contracts](https://bigquery.cloud.google.com/dataset/gpqueries:contracts)

*Important:* You need a Google account and a Google Cloud project to access the data (both free). Google offers you to create a new project when you open BigQuery. Do it. Then you'll need to follow Google's instructions and enable BigQuery in this project.

## Table `gpqueries:contracts.raw`

Table [`gpqueries:contracts.raw`](https://bigquery.cloud.google.com/table/gpqueries:contracts.raw) contains the unmodified data from the [USASpending.gov archives](https://www.usaspending.gov/DownloadCenter/Pages/dataarchives.aspx). It's constructed from `<year>_All_Contracts_Full_20160515.csv.zip` files and includes contracts from 2000 to May 15, 2016.

Table `gpqueries:contracts.raw` contains 45M rows and 225 columns.

Each row refers to a transaction (a purchase or refund) made by a federal agency. It may be a pizza or an airplane.

The columns are grouped into categories:

- Transaction: `unique_transaction_id`-`baseandalloptionsvalue`
- Buyer (government agency): `maj_agency_cat`-`fundedbyforeignentity`
- Dates: `signeddate`-`lastdatetoorder`, `last_modified_date`
- Contract: `contractactiontype`-`programacronym`
- Contractor (supplier, vendor): `vendorname`-`statecode`
- Place of performance: `PlaceofPerformanceCity`-`placeofperformancecongressionaldistrict`
- Product or service bought: `psc_cat`-`manufacturingorganizationtype`
- General contract information: `agencyid`-`idvmodificationnumber`
- Competitive procedure: `solicitationid`-`statutoryexceptiontofairopportunity`
- Contractor details: `organizationaltype`-`otherstatutoryauthority`
- Contractor's executives: `prime_awardee_executive1`-`interagencycontractingauthority`

Detailed description for each variable is available in the official codebook:

- [`USAspending.govDownloadsDataDictionary.pdf`](https://www.usaspending.gov/DownloadCenter/Documents/USAspending.govDownloadsDataDictionary.pdf)

## Queries

### BigQuery Web GUI

You can execute queries mentioned here at <https://bigquery.cloud.google.com/table/gpqueries:contracts.raw> (press "Compose query").

### Datalab

This notebook was written in Google Datalab. You may need the libraries imported below to replicate it:

In [1]:
import gcp.bigquery as bq

Within Datalab, you can define queries with [`sql` magic](https://github.com/catherinedevlin/ipython-sql) like this:

In [2]:
%%sql --module gpq

define query totals
select
  count(*) transactions,
  sum(dollarsobligated) sum_dollarsobligated,
  count(unique(dunsnumber)) vendors,
  count(unique(solicitationid)) purchase_procedures
from 
  gpqueries:contracts.raw

And execute with `bq` to get a dataframe:

In [3]:
bq.Query(gpq.totals).to_dataframe()

,transactions,sum_dollarsobligated,vendors,purchase_procedures
0,44500941,6.653578e+12,621532,2223674


Which means we're dealing with 44.5M transactions totalling 6.7 trillion dollars. These purchases came from 622k vendors that won 2.2mn solicitations issued by government agencies.

## Data Mining Government Clients

Suppose you want to start selling to the government. While [FBO.gov](http://www.fbo.gov/) publishes government RFPs and you can apply there, government agencies often issue requests when they've already chosen the supplier. Agencies go through FBO.gov because it's a mandatory step for deals north of $25K. But winning at this stage is unlikely if an RFP is already tailored for another supplier.

Reaching warm leads in advance would increase chances of winning a government contract. The contracts data helps identify the warm leads by looking at purchases in the previous years.

There're several ways of searching through those years.

### Who Buys What You Make

The goods and services bought in each transaction are encoded in the variable `productorservicecode`. Top ten product categories according to this variable:

In [4]:
%%sql
select
  substr(productorservicecode, 1, 4) product_id,
  first(substr(productorservicecode, 7)) product_name,
  count(*) transactions,
  sum(dollarsobligated) sum_dollarsobligated
from
  gpqueries:contracts.raw
group by
  product_id
order by
  sum_dollarsobligated desc
limit 10

product_id,product_name,transactions,sum_dollarsobligated
1510,"AIRCRAFT, FIXED WING",34373,2.84778910095e+11
M181,OPER OF GOVT R&D GOCO FACILITIES,11705,2.09034830695e+11
R425,ENGINEERING AND TECHNICAL SERVICES,540128,2.01977381246e+11
D399,OTHER ADP & TELECOMMUNICATIONS SVCS,544069,1.79749708292e+11
R499,OTHER PROFESSIONAL SERVICES,821788,1.63569940538e+11
9130,LIQUID PROPELLANTS -PETROLEUM BASE,193267,1.47092818248e+11
Q201,GENERAL HEALTH CARE SERVICES,157188,1.43893967601e+11
6505,DRUGS AND BIOLOGICALS,1857244,1.40279701467e+11
R706,LOGISTICS SUPPORT SERVICES,147238,1.17202582504e+11
R408,PROGRAM MANAGEMENT/SUPPORT SERVICES,270904,89610029462.5


You can find agencies that buy products like yours. If it's "software":

In [5]:
%%sql
select
  substr(agencyid, 1, 4) agency_id,
  first(substr(agencyid, 7)) agency_name,
  count(*) transactions,
  sum(dollarsobligated) sum_dollarsobligated
from
  gpqueries:contracts.raw
where
  productorservicecode contains 'software'
group by
  agency_id
order by
  sum_dollarsobligated desc
ignore case

agency_id,agency_name,transactions,sum_dollarsobligated
9700,DEPT OF DEFENSE,245015,31756394755.4
3600,"VETERANS AFFAIRS, DEPARTMENT OF",21223,3213678011.54
2050,INTERNAL REVENUE SERVICE,13292,2625458725.97
2800,SOCIAL SECURITY ADMINISTRATION,7038,2383445352.45
4735,FEDERAL ACQUISITION SERVICE,15064,2374494872.16
7014,U.S. CUSTOMS AND BORDER PROTECTION,3456,1816831446.67
1900,"STATE, DEPARTMENT OF",14964,1395765461.73
1549,FEDERAL BUREAU OF INVESTIGATION,4673,1104923392.94
7001,OFFICE OF PROCUREMENT OPERATIONS,2604,1076395157.89
1501,"OFFICES, BOARDS AND DIVISIONS",4617,1057842936.58


### What Firms in Your Industry Sell to the Government

Another way to find customers is the variable `principalnaicscode` that encodes the industry in which the vendor does business.

The list of NAICS codes is available at [Census.gov](http://www.census.gov/cgi-bin/sssd/naics/naicsrch?chart=2012), but you can do text search in the table. Let's find who bought software from distributors in 2015:

In [6]:
%%sql
select
  substr(agencyid, 1, 4) agency_id,
  first(substr(agencyid, 7)) agency_name,
  substr(principalnaicscode, 1, 6) naics_id,
  first(substr(principalnaicscode, 9)) naics_name,
  count(*) transactions,
  sum(dollarsobligated) sum_dollarsobligated
from
  gpqueries:contracts.raw
where
  principalnaicscode contains 'software' and
  fiscal_year = 2015
group by
  agency_id, naics_id
order by
  sum_dollarsobligated desc
ignore case

agency_id,agency_name,naics_id,naics_name,transactions,sum_dollarsobligated
9700,DEPT OF DEFENSE,511210,SOFTWARE PUBLISHERS,7907,1086588808.06
9700,DEPT OF DEFENSE,443120,COMPUTER AND SOFTWARE STORES,5313,451131013.69
9700,DEPT OF DEFENSE,423430,COMPUTER AND COMPUTER PERIPHERAL EQUIPMENT AND SOFTWARE MERCHANT WHOLESALERS,2073,246620960.81
7014,U.S. CUSTOMS AND BORDER PROTECTION,443120,COMPUTER AND SOFTWARE STORES,294,195178029.07
2800,SOCIAL SECURITY ADMINISTRATION,443120,COMPUTER AND SOFTWARE STORES,201,165682384.14
2050,INTERNAL REVENUE SERVICE,443120,COMPUTER AND SOFTWARE STORES,272,105303702.28
1549,FEDERAL BUREAU OF INVESTIGATION,443120,COMPUTER AND SOFTWARE STORES,147,94253371.19
1900,"STATE, DEPARTMENT OF",443120,COMPUTER AND SOFTWARE STORES,1196,91138323.46
2800,SOCIAL SECURITY ADMINISTRATION,511210,SOFTWARE PUBLISHERS,244,89509652.8
3600,"VETERANS AFFAIRS, DEPARTMENT OF",511210,SOFTWARE PUBLISHERS,630,87867702.88


### Inspecting Specific Transactions

You can learn details from looking at transactions for a specific `(agency, NAICS)` pair. For example, what software does TSA buy?

In [7]:
%%sql
select
  fiscal_year,
  dollarsobligated,
  vendorname, city, state, annualrevenue, numberofemployees,
  descriptionofcontractrequirement
from
  gpqueries:contracts.raw
where
  agencyid contains 'transportation security administration' and
  principalnaicscode contains 'computer and software stores'
ignore case

fiscal_year,dollarsobligated,vendorname,city,state,annualrevenue,numberofemployees,descriptionofcontractrequirement
2003,75480.0,INTERNATIONAL BUSINESS MACHINE,BETHESDA,MD,0.0,0,
2003,64800.0,AVAYA INC,ARLINGTON,VA,0.0,0,
2016,0.0,"IMMIXTECHNOLOGY, INC.",MCLEAN,VA,31929440.0,162,CLOSEOUT SERVICE
2008,5483.8,"SPECTRUM SYSTEMS, INC.",FAIRFAX,VA,17000000.0,25,"IT SECURITY PR TO FUND ANNUAL LICENSE RENEWAL FOR SPIDYNAMICS WEBINSPECT IN THE AMOUNT OF $4,500 PR AMENDED- TO MATCH LOWEST QUOTE AND TO EXTEND SERVICE THROUGH 12/31/2009."
2007,557880.0,GTSI CORPORATION,CHANTILLY,VA,900000000.0,850,THIS PROCUREMENT REQUEST IS FOR 250 D620 TRACE MACHINES FOR THE FSDS PER THE ATTACHED CONFIGURATION. DELIVERY SCHEDULE AS FOLLOWS: DEC 18 - 25 JAN 8 - 25 JAN 15 - 50 JAN 29 - 50 FEB 12 - 50 FEB 26 - 50
2004,1640913.0,INTERNATIONAL BUSINESS MACHINE,BETHESDA,MD,0.0,0,IT SERVICES
2013,36372.5,"IMMIXTECHNOLOGY, INC.",MCLEAN,VA,31929440.0,162,THE PURPOSE OF THIS BPA CALL IS TO PURCHASE A QUANTITY OF 250 POE WITH BATTERY BACKUP (PART # 8602806-001)TERMINAL CLOCK PRODUCTS.
2013,5373123.87,INTERNATIONAL BUSINESS MACHINE,BETHESDA,MD,0.0,0,IGF::OT::IGF SUBJECT: APPLICATION SUPPORT TEAM (AST) - OASIS II WORK ORDER
2008,16077.46,SPRINT COMMUNICATIONS COMPANY L.P.,RESTON,VA,38927998976.0,56001,QSEC AIRTIME FOR A BASE AND TWO ADDITIONAL OPTION PERIODS.
2005,100000.0,ORACLE CORPORATION,RESTON,VA,0.0,0,AFSP ORACLE SUPPORT


Alternatively, specify vendors your product relates to and check how the government uses it. Top deals in data analytics:

In [8]:
%%sql
select
  agencyid,
  dollarsobligated,
  vendorname,
  descriptionofcontractrequirement
from
  gpqueries:contracts.raw
where
  vendorname contains 'tableau' or
  vendorname contains 'socrata' or
  vendorname contains 'palantir' or
  vendorname contains 'revolution analytics' or
  vendorname contains 'mathworks' or
  vendorname contains 'statacorp' or
  vendorname contains 'mathworks'
order by
  dollarsobligated desc
limit
  100
ignore case

agencyid,dollarsobligated,vendorname,descriptionofcontractrequirement
9700: DEPT OF DEFENSE,22401901.14,PALANTIR TECHNOLOGIES INCORPORATED,PROCUREMENT OF PALANTIR GOTHAM SOFTWARE AND ASSOCIATED SUPPORT
9700: DEPT OF DEFENSE,19243057.44,PALANTIR TECHNOLOGIES INCORPORATED,PALANTIR CORE SERVER LICENSES AND HARDWARE
1549: FEDERAL BUREAU OF INVESTIGATION,14891225.66,PALANTIR TECHNOLOGIES INC.,UNLIMITED LICENSE
7012: U.S. IMMIGRATION AND CUSTOMS ENFORCEMENT,12472821.2,"PALANTIR USG, INC.","ICE COTS BASED, WEB-ENABLED INVESTIGATIVE CASE MANAGEMENT (ICM) SYSTEM FOR ICE TECS MOD PROGRAM OFFICE. IGF::CT::IGF"
9700: DEPT OF DEFENSE,11999886.24,PALANTIR TECHNOLOGIES INCORPORATED,IGF::OT::IGF EXERCISE OPTION YEAR 1 AND ADD FUNDING
9700: DEPT OF DEFENSE,11805943.2,PALANTIR TECHNOLOGIES INCORPORATED,PALANTIR SW LICENSE MAINTENANCE SUPPORT RENEWAL.
7012: U.S. IMMIGRATION AND CUSTOMS ENFORCEMENT,9900000.0,PALANTIR TECHNOLOGIES INC.,"IGF::OT::IGF FALCON OPERATIONS AND MAINTENANCE (O&M), SYSTEM ENHANCEMENT SUPPORT SERVICES FOR PALANTIR GOVERNMENT"
7012: U.S. IMMIGRATION AND CUSTOMS ENFORCEMENT,9857193.77,"PALANTIR USG, INC.","ICE COTS BASED, WEB-ENABLED INVESTIGATIVE CASE MANAGEMENT (ICM) SYSTEM FOR ICE TECS MOD PROGRAM OFFICE. IGF::CT::IGF"
"1501: OFFICES, BOARDS AND DIVISIONS",9500000.0,PALANTIR TECHNOLOGIES INC.,IGF::OT::IGF
9700: DEPT OF DEFENSE,9125264.2,PALANTIR TECHNOLOGIES INCORPORATED,IFAP


### Searching Through Descriptions

Full-text search and regular expressions for the variable `descriptionofcontractrequirement` narrow results for relevant product groups:

In [9]:
%%sql
select
  agencyid,
  dollarsobligated,
  descriptionofcontractrequirement
from
  gpqueries:contracts.raw
where
  descriptionofcontractrequirement contains 'body camera'
limit
  100
ignore case

agencyid,dollarsobligated,descriptionofcontractrequirement
9700: DEPT OF DEFENSE,5529.0,WEARABLE BODY CAMERA
1549: FEDERAL BUREAU OF INVESTIGATION,3978.0,BODY CAMERAS FOR WARM SPRINGS OFFICERS/DETECTIVES
7008: U.S. COAST GUARD,5154.21,POLICE BODY CAMERAS
1443: NATIONAL PARK SERVICE,12044.42,CACO WEARABLE BODY CAMERAS FOR THE LAW ENFORCEMENT RANGERS AT CAPE CODE NATIONAL SEASHORE
9700: DEPT OF DEFENSE,4315.0,NIKON D3 DIGITAL SLR BODY CAMERA
9700: DEPT OF DEFENSE,26285.6,BODY CAMERA'S
1443: NATIONAL PARK SERVICE,14605.4,VIEVU BODY CAMERAS
2036: BUREAU OF THE FISCAL SERVICE,23970.0,BODY CAMERAS
1443: NATIONAL PARK SERVICE,14850.0,LARO LE BODY CAMERAS
"3600: VETERANS AFFAIRS, DEPARTMENT OF",5648.0,16 BODY CAMERAS FOR POLICE DEPT AT LEXINGTON VAMC


Some rows of `descriptionofcontractrequirement` contain codes like "IGF::CT::IGF". These codes classify the purchase into three groups of "[Inherently Governmental Functions](https://www.fpds.gov/fpdsng_cms/index.php/en/newsroom/108-nherently-governmental-functions.html)" (IGF):

1. IGF::CT::IGF for Critical Functions
2. IGF::CL::IGF for Closely Associated
3. IGF::OT::IGF for Other Functions

### Narrowing Your Geography

You can find local opportunities using variables for vendors (`city`, `state`) and services sold (`PlaceofPerformanceCity`, `pop_state_code`). The states where most contracts are delivered in:

In [10]:
%%sql
select
  substr(pop_state_code, 1, 2) state_code,
  first(substr(pop_state_code, 4)) state_name,
  sum(dollarsobligated) sum_dollarsobligated
from
  gpqueries:contracts.raw
group by
  state_code
order by
  sum_dollarsobligated desc

state_code,state_name,sum_dollarsobligated
CA,,7.12598903911e+11
VA,VIRGINIA,6.97817723705e+11
:,,6.67791793514e+11
TX,TEXAS,5.12968110673e+11
MD,MARYLAND,3.51494588833e+11
DC,DISTRICT OF COLUMBIA,2.58102498463e+11
PA,PENNSYLVANIA,2.10745481833e+11
FL,FLORIDA,2.10241041129e+11
MA,MASSACHUSETTS,1.84166810727e+11
AZ,ARIZONA,1.68027358301e+11


## Facts about Government Contracting

Let's check some popular statements about government contracting.

### Small Businesses Win Most Contracts

Contractors had to report their revenue and the number of employees. It makes easy to check if small business is welcomed in government contracting:

In [11]:
%%sql --module gpq

define query vendor_size_by_agency
select
  substr(agencyid, 1, 4) agency_id,
  first(substr(agencyid, 7)) agency_name,
  nth(11, quantiles(annualrevenue, 21)) vendor_median_annualrevenue,
  nth(11, quantiles(numberofemployees, 21)) vendor_median_numberofemployees,
  count(*) transactions,
  sum(dollarsobligated) sum_dollarsobligated
from
  gpqueries:contracts.raw
group by
  agency_id
having
  transactions > 1000 and
  sum_dollarsobligated > 10e6
order by
  vendor_median_annualrevenue asc

In [12]:
bq.Query(gpq.vendor_size_by_agency).to_dataframe()

,agency_id,agency_name,vendor_median_annualrevenue,vendor_median_numberofemployees,transactions,sum_dollarsobligated
0,1242,ANIMAL AND PLANT HEALTH INSPECTION SERVICE,0.0,0,1268,1.418465e+08
1,1528,IMMIGRATION AND NATURALIZATION SERVICE,0.0,0,4858,1.808390e+09
2,5800,FEDERAL EMERGENCY MANAGEMENT AGENCY,0.0,0,3857,7.360614e+08
3,1900,"STATE, DEPARTMENT OF",1000.0,10,906835,9.612514e+10
4,9568,BROADCASTING BOARD OF GOVERNORS,1000.0,1,86580,1.719051e+09
5,7200,AGENCY FOR INTERNATIONAL DEVELOPMENT,1000.0,10,112903,5.683631e+10
6,2800,SOCIAL SECURITY ADMINISTRATION,81000.0,1,140669,1.636968e+10
7,1226,FOREST SERVICE,100000.0,2,24686,1.554254e+09
8,12E3,RURAL HOUSING SERVICE,110000.0,2,87659,1.124532e+09
9,2028,U.S. SECRET SERVICE,110000.0,2,1144,1.870609e+08


The median shows the most likely supplier. Agencies on the top of the table actively employ vendors whose annual revenue is less than $1mn.

The Department of Defence, the largest buyer with $4.5tn worth of goods and services bought over these 17 years, has the median vendor with $2.5mn in revenue and 20 employees. It means that half of the DoD's vendors have less than $2.5mn in revenue.

### Set-Aside Deals Take a Small Share

Set-aside purchases are reserved for special categories of suppliers, like women-, minority-, and veteran-owned businesses. There's a lot of confusion about their share in transactions. We can settle this confusion with data:

In [13]:
%%sql
select
  womenownedflag,
  count(*) transactions,
  sum(dollarsobligated) sum_dollarsobligated
from
  gpqueries:contracts.raw
group by
  womenownedflag

womenownedflag,transactions,sum_dollarsobligated
N,40191975,6.4051823538e+12
Y,4308966,2.48395258992e+11


Women-owned businesses make about one tenth of the transactions, but their share in terms of sales is only 3.7%.

A cross-tabulation for major set-aside categories:

In [14]:
%%sql
select
  womenownedflag, veteranownedflag, minorityownedbusinessflag,
  count(*) transactions,
  sum(dollarsobligated) sum_dollarsobligated
from
  gpqueries:contracts.raw
group by
  womenownedflag, veteranownedflag, minorityownedbusinessflag
order by
  womenownedflag, veteranownedflag, minorityownedbusinessflag desc

womenownedflag,veteranownedflag,minorityownedbusinessflag,transactions,sum_dollarsobligated
N,N,Y,2051978,2.70617792041e+11
N,N,N,35286333,5.91438262215e+12
N,Y,Y,494617,61028950046.2
N,Y,N,2359047,1.59152989569e+11
Y,N,Y,862336,80295343639.0
Y,N,N,3265163,1.53867438135e+11
Y,Y,Y,62597,5086157754.65
Y,Y,N,118870,9146319463.28


For example, firms owned by women, veterans, and minorities (all represented at the same time) sell $5bn in goods and services. That's 0.07% of all government purchases.

### New Vendors Emerge Each Year

Becoming a government contractor may seem difficult at first, but let's see how many new contractors the government had in 2015.

In [15]:
%%sql
select
  sum(if(before2015.dunsnumber is null, 1, 0)) new_vendors,
  sum(if(before2015.dunsnumber is null, 0, 1)) old_vendors
from
  flatten((select unique(dunsnumber) dunsnumber from gpqueries:contracts.raw where fiscal_year = 2015), dunsnumber) in2015
left join
  flatten((select unique(dunsnumber) dunsnumber from gpqueries:contracts.raw where fiscal_year < 2015), dunsnumber) before2015
  on before2015.dunsnumber = in2015.dunsnumber

new_vendors,old_vendors
18258,131537


(The variable `dunsnumber` refers to the unique number each contractor obtains at [SAM.gov](https://www.sam.gov/portal/SAM/). It identifies unique contractors better than `vendorname`, which varies.)

In 2015, 12% of the suppliers turned out to be companies that had never sold to the government.

## More

This was a short intro to government contracting data.

For more, check the [Government Procurement Queries](https://github.com/antontarasenko/gpq) project.

## Contacts

Anton Tarasenko

<mailto:antontarasenko@gmail.com>

<http://antontarasenko.com>

